In [9]:
using JuMP, GLPKMathProgInterface, Clp, Cbc
#using CPLEX
using MathProgBase

In [3]:
#numbers = rand(5000,5000);
#writedlm("test.txt", numbers)
#numbers = readdlm("test.txt");

In [4]:
function crearDict(conjunto)
    cont = 1;
    dict = Dict();
    for i in conjunto
        dict[i] = cont
        cont +=1;
    end
    return dict
end

crearDict (generic function with 1 method)

In [5]:
## MAIN SETS
include("Conjuntos_ses_main.jl")

Dict{String,Array{String,1}} with 5 entries:
  "ELECTRICITY"        => String["NUCLEAR", "CCGT", "CCGT_CCS", "COAL_US", "COA…
  "MOBILITY_PASSENGER" => String["TRAMWAY_TROLLEY", "BUS_COACH_DIESEL", "BUS_CO…
  "HEAT_HIGH_T"        => String["IND_COGEN_GAS", "IND_COGEN_WOOD", "IND_COGEN_…
  "HEAT_LOW_T"         => String["DHN_HP_ELEC", "DHN_COGEN_GAS", "DHN_COGEN_WOO…
  "MOBILITY_FREIGHT"   => String["TRAIN_FREIGHT", "TRUCK"]

In [6]:
LAYERSD = crearDict(LAYERS);
TECHNOLOGIESD = crearDict(TECHNOLOGIES);
RESOURCESD = crearDict(RESOURCES);
TECHNOLOGIESdiffINFRASTRUCTURED = crearDict(setdiff(TECHNOLOGIES,INFRASTRUCTURE));
TECHNOLOGIES_OF_END_USES_TYPEHDD = crearDict(setdiff(TECHNOLOGIES_OF_END_USES_TYPE["HEAT_LOW_T_DECEN"],["DEC_SOLAR"]));
STORAGE_TECHD = crearDict(STORAGE_TECH);
END_USES_TYPESD = crearDict(END_USES_TYPES);
op_strategy_mob_privateD = crearDict(union(TECHNOLOGIES_OF_END_USES_CATEGORY["MOBILITY_PASSENGER"],TECHNOLOGIES_OF_END_USES_CATEGORY["MOBILITY_FREIGHT"]));

In [7]:
## PARAMETERS
include("datos_ses_main.jl")

8760

In [10]:
## leer modelo
include("modelo_ses_main.jl");

In [11]:
#include("solucion.jl")

In [12]:
#include("factibilidad.jl")
#factibilidad(6)

In [13]:
status = solve(modelo)
getobjectivevalue(modelo)

13668.185984877602

In [14]:
LinearConstraint(storage_level[1,1])

F_Mult_t[POWER2GAS,1] - F_Mult_t[POWER2GAS,12] - 744 Storage_In[POWER2GAS,LNG,1] + 744 Storage_Out[POWER2GAS,LNG,1] = 0

In [15]:
println(getdual(totalGWP_calc))

NaN


In [20]:
#f = open("model.lp", "w")
#print(f, modelo)
#close(f)

In [19]:
f = open("solucion.txt", "w")
for i in TECHNOLOGIES
    println(f,"let Number_Of_Units['$(i)'] := $(getvalue(Number_Of_Units[i]));")
    println(f,"let F_Mult['$(i)'] := $(getvalue(F_Mult[i]));")
    println(f,"let C_inv['$(i)'] := $(getvalue(C_inv[i]));")
    println(f,"let C_maint['$(i)'] := $(getvalue(C_maint[i]));")
    println(f,"let Y_Solar_Backup['$(i)'] := $(getvalue(Y_Solar_Backup[i]));")
    println(f,"let GWP_constr['$(i)'] := $(getvalue(GWP_constr[i]));")
end

for i in LAYERS, t in PERIODS
    println(f,"let End_Uses['$(i)',$(t)] := $(getvalue(End_Uses[i,t]));")
end

for i in union(RESOURCES,TECHNOLOGIES), t in PERIODS
    println(f,"let F_Mult_t['$(i)',$(t)] := $(getvalue(F_Mult_t[i,t]));")
end

for i in RESOURCES
    println(f,"let C_op['$(i)'] := $(getvalue(C_op[i]));")
end

for in i STORAGE_TECH, j in LAYERS, t in PERIODS
    println(f,"Storage_In['$(i)','$(j)',$(t)] := $(getvalue(Storage_In[i,j,t]));")

close(f)

In [ ]:
MathProgBase.numvar(modelo) # devuelve el numero de variables
MathProgBase.numlinconstr(modelo) # devuelve el numero de restricciones lineales